### Questions regarding Qiskit Runtime ###

1) What exactly makes runtime faster? Is it just that the classical compute also happens closer to the quantum compute in the cloud, or is there something more? Does qiskit runtime change the ordering of jobs too? That is, if someone starts a runtime session and the first job is executed, will the other jobs be given priority? If that is the case, what happens if multiple people are running a runtime session on the same backend?

2) Why is the sampler output called "quasi-probability"? Why is it not real probability? How is a sampler primitive different from say the qasm-simulator?

3) Does the estimator do something fundamentally different that makes it faster? Like in order to compute the expectations (on a real device, or qasm simulator), we will still have to get shots. What does it do in the background? Does it simply generate those shots and compute expectations manually? Or is there some fundamentally different technique? 

### Primitives ###

Runtime has a couple of new runtime primitives, that enable the user to do a few additional tasks (on top of what is normally possible using qiskit)

### Sampler ###

Takes as input a circuit, returns error-mitigated quasi-probabilites as output.

### Estimator ###

Efficiently calculate expectation values of certain operators

In [ ]:
import qiskit
from qiskit_ibm_provider import IBMProvider

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

In [ ]:
service = QiskitRuntimeService(channel="ibm_quantum")

In [ ]:
from qiskit import *
from qiskit_ibm_runtime import Sampler, Estimator

In [ ]:
service = QiskitRuntimeService()
#backend = service.get_backend("ibmq_qasm_simulator")

qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0,1)
qc.measure_all()

sampler = Sampler(session=backend)
job = sampler.run(qc)
result = job.result()

### Estimator Primitive Tutorial ###

In [ ]:
from qiskit.circuit.random import random_circuit

circuit = random_circuit(2, 2, seed = 0).decompose(reps = 1)

In [ ]:
circuit.draw(output = "mpl")

### Define an observable ###

In [ ]:
from qiskit.quantum_info import SparsePauliOp

observable = SparsePauliOp("XZ")
print(f">>> Observable: {observable.paulis}")

### Initialize an Estimator object ###

In [ ]:
from qiskit.primitives import Estimator, Sampler

In [ ]:
estimator = Estimator()
job = estimator.run(circuit, observable)

In [ ]:
result = job.result()

### Multiple Circuits ###

In [ ]:
circuits = (
    random_circuit(2, 2, seed=0).decompose(reps=1),
    random_circuit(2, 2, seed=1).decompose(reps=1),
)
observables = (
    SparsePauliOp("XZ"),
    SparsePauliOp("IY"),
)

job = estimator.run(circuits, observables)
result = job.result()

print(f">>> Observables: {[obs.paulis for obs in observables]}")
print(f">>> Expectation values: {result.values.tolist()}")

### Runtime Estimator ###

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(channel="ibm_quantum")
backend = service.backend("ibmq_qasm_simulator") # the simulator up on the cloud

In [ ]:
from qiskit.circuit.random import random_circuit
from qiskit.quantum_info import SparsePauliOp

circuit = random_circuit(2, 2, seed=0).decompose(reps=1)
display(circuit.draw("mpl"))

observable = SparsePauliOp("XZ")
print(f">>> Observable: {observable.paulis}")

In [ ]:
from qiskit_ibm_runtime import Estimator, Sampler

In [ ]:
estimator = Estimator(backend = backend) # estimator primitive on top of the qasm_simulator

In [ ]:
job = estimator.run(circuit, observable)
print(f">>> Job ID: {job.job_id()}")
print(f">>> Job Status: {job.status()}")

In [ ]:
result = job.result()
print(f">>> {result}")
print(f"  > Expectation value: {result.values[0]}")
print(f"  > Metadata: {result.metadata[0]}")

In [ ]:
from qiskit_ibm_runtime import Options

options = Options(optimization_level = 3, environment = {"log_level": "INFO"})

### optimization_level for error suppression and resilience_level for error mitigation ###

In [ ]:
options = Options(optimiztion_level = 3, resilience_level = 3)

### Invoke Estimator within a session ###

In [ ]:
backend = service.backend("ibmq_qasm_simulator")

In [ ]:
from qiskit_ibm_runtime import Session, Estimator

with Session(backend = backend, max_time = "1h"):
    
    options = Options(optimization_level = 3)
    estimator = Estimator(options = options)
    
    for i in range(100):
        result = estimator.run(circuit, observable).result()
        print(f">>> Expectation value from the {i}-th run: {result.values[0]}")

### Overall Target To-do ###

1) Get Initial Parameter estimate using CAFQA
2) VAQEM+QISMET+VarSaw (DD insertion, Transient Errors, Measurement Errors)

### Week 1 : To do ###

1) Complete reading all qiskit runtime tutorials
2) Run all the circuit examples on a simulator, as well as real hardware
3) Try to run VQE with runtime along with VarSaw
4) Read the VAQEM paper

### VQE with Estimator ###

In [ ]:
# General imports
import time
import numpy as np

# Pre-defined ansatz circuit and operator class for Hamiltonian
from qiskit.circuit.library import EfficientSU2
from qiskit.quantum_info import SparsePauliOp

# The IBM Qiskit Runtime
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Estimator, Session

# SciPy minimizer routine
from scipy.optimize import minimize

# Plotting functions
import matplotlib.pyplot as plt

%config InlineBackend.figure_format='retina'

In [ ]:
service = QiskitRuntimeService()
backend = service.get_backend("ibmq_qasm_simulator")
#backend = service.get_backend("ibmq_mumbai")

In [ ]:
hamiltonian = SparsePauliOp.from_list(
    [("YZ", 0.3980), ("ZI", -0.3980), ("ZZ", -0.0113), ("XX", 0.1810)]
)

In [ ]:
ansatz = EfficientSU2(hamiltonian.num_qubits)
ansatz.draw("mpl")

In [ ]:
num_params = ansatz.num_parameters
print(num_params)

In [ ]:
# vqe cost function

def cost_func(params, ansatz, hamiltonian, estimator):
    """Return estimate of energy from estimator

    Parameters:
        params (ndarray): Array of ansatz parameters
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        estimator (Estimator): Estimator primitive instance

    Returns:
        float: Energy estimate
    """
    energy = (
        estimator.run(ansatz, hamiltonian, parameter_values=params).result().values[0]
    )
    print(energy)
    return energy

In [ ]:
x0 = 2 * np.pi * np.random.random(num_params)

In [ ]:
with Session(backend=backend):
    estimator = Estimator(options={"shots": int(1e4)})
    res = minimize(
        cost_func, x0, args=(ansatz, hamiltonian, estimator), method="cobyla"
    )

### Runtime Sampler ###

In [ ]:
from qiskit import QuantumCircuit

In [ ]:
from qiskit.circuit.random import random_circuit

#circuit = random_circuit(2, 2, seed=0, measure=True).decompose(reps=1)
circuit = QuantumCircuit(2, 2)
circuit.h(0)
circuit.cx(0, 1)
circuit.measure([0, 1], [0, 1])
display(circuit.draw("mpl"))

In [ ]:
from qiskit.primitives import Sampler

sampler = Sampler()

In [ ]:
job = sampler.run(circuit)
print(f">>> Job ID: {job.job_id()}")
print(f">>> Job Status: {job.status()}")

In [ ]:
result = job.result()
print(f">>> {result}")
print(f"  > Quasi-distribution: {result.quasi_dists[0]}")

### Mitiq testing ###

In [5]:
USE_REAL_HARDWARE = True

In [ ]:
def vqe_derivative()